### Пример спам-фильтра

In [ ]:
# creating word dictionary
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]    
    all_words = []       
    for mail in emails:    
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    
    list_to_remove = list(dictionary.keys())
    for item in list_to_remove:
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    return dictionary

In [ ]:
def extract_features(mail_dir): 
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    docID = 0;
    for fil in files:
        with open(fil) as fi:
            for i,line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                            wordID = 0
                            for i,d in enumerate(dictionary):
                                if d[0] == word:
                                    wordID = i
                                    features_matrix[docID,wordID] = words.count(word)
            docID = docID + 1     
    return features_matrix

In [ ]:
# training the classifiers
import os
import numpy as np
from collections import Counter

In [ ]:
# Create a dictionary of words with its frequency
train_dir = 'train-mails'
dictionary = make_Dictionary(train_dir)

In [ ]:
type(dictionary[0])

tuple

In [ ]:
print(dictionary[:5])

[('order', 1414), ('address', 1293), ('report', 1216), ('mail', 1127), ('send', 1079)]


In [ ]:
# Prepare feature vectors per training mail and its labels
y_train = np.zeros(702) # y_train
y_train[351:701] = 1
train_matrix = extract_features(train_dir)
# Bag Of Words
# I like machine learning

#dictionary: 
#i - 1, like - 2, dog - 3, cat - 4, machine - 5, learning - 6
# (I like machine learning) ----->>>>>>> (1,1,0,0,1,1)
# (I like machine learning and i am too) ------->>>>> (2,1,0,0,1,1)


In [ ]:
print(type(train_matrix),train_matrix.shape) 

<class 'numpy.ndarray'> (702, 3000)


In [ ]:
# Training SVM and Naive bayes classifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 10

model1 = MultinomialNB()
model2 = KNeighborsClassifier(n_neighbors)
model1.fit(train_matrix,y_train)
model2.fit(train_matrix,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [ ]:
# Test the unseen mails for Spam
test_dir = 'test-mails'
test_matrix = extract_features(test_dir)
y_test = np.zeros(260)
y_test[130:260] = 1

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
result1 = model1.predict(test_matrix)
result2 = model2.predict(test_matrix)
print(confusion_matrix(y_test,result1), accuracy_score(y_test,result1))
print(confusion_matrix(y_test,result2), accuracy_score(y_test,result2))

[[86 44]
 [77 53]] 0.5346153846153846
[[81 49]
 [85 45]] 0.4846153846153846
